In [ ]:
import pandas as pd
import numpy as np
import os
import json

In [ ]:
import io
import extract_msg
from PyPDF2 import PdfReader  # Для работы с PDF
from typing import Tuple  # Для типизации

In [ ]:
# #pdf2txt
# import io
# from pdfminer.converter import TextConverter
# from pdfminer.pdfinterp import PDFPageInterpreter
# from pdfminer.pdfinterp import PDFResourceManager
# from pdfminer.pdfpage import PDFPage

In [79]:
# Вытаскиваем PathSupplierLetter – путь к письму поставщика на ПК
# folder_path = 'C:\Users\sokolov\Yandex_Disk\MyData\Projects\loading_into_brand_card_iek\exchange\'
folder_path = 'C:/Users/sokolov/Yandex_Disk/MyData/Projects/loading_into_brand_card_iek/3/'
for file in os.listdir(folder_path):
    if '.msg' in file:
        PathSupplierLetter = folder_path + file
print(PathSupplierLetter)

C:/Users/sokolov/Yandex_Disk/MyData/Projects/loading_into_brand_card_iek/3/Условия по тендеру Т250721-38067 согласованы IEKCRM0002684.msg


In [88]:
def process_msg_file(PathSupplierLetter: str) -> Tuple[pd.DataFrame, str]:
    """
    Обрабатывает файл .msg, извлекает Excel и PDF вложения,
    преобразует Excel в DataFrame, PDF в текст.
    
    Args:
        msg_path (str): Путь к файлу .msg
        
    Returns:
        Tuple[pd.DataFrame, str]: DataFrame из Excel и текст из PDF
    """
    excel_df = None
    pdf_text = ""
    
    try:
        # Открываем файл .msg
        msg = extract_msg.Message(PathSupplierLetter)
        
        # Проверяем, что есть ровно 2 вложения
        if len(msg.attachments) != 2:
            raise ValueError("В письме должно быть ровно 2 вложения")
            
        for attachment in msg.attachments:
            filename = attachment.longFilename
            
            # Обработка Excel файла
            if filename.lower().endswith('.xlsx'):
                excel_data = io.BytesIO(attachment.data)
                specification_xlsx = pd.read_excel(excel_data)
                
            # Обработка PDF файла
            elif filename.lower().endswith('.pdf'):
                pdf_data = io.BytesIO(attachment.data)
                pdf_reader = PdfReader(pdf_data)
                
                # Собираем текст со всех страниц
                for page in pdf_reader.pages:
                    pdf_text += page.extract_text() + "\n"
                    
            else:
                raise ValueError(f"Неподдерживаемый формат файла: {filename}")
                
        msg.close()
        
    except Exception as e:
        print(f"Произошла ошибка: {str(e)}")
        raise
        
    return specification_xlsx, pdf_text

In [93]:
specification_xlsx, pdf_text = process_msg_file(PathSupplierLetter)
text = pdf_text.strip()
text = text.replace('\n', ' ')
text = text.replace('    ', ' ')
text = text.replace('   ', ' ')
text = text.replace('  ', ' ')
text

'Уважаемые Партнеры! В ответ на запрос о предоставлении разовой скидки компании Алекс, ООО ИНН 5410082888 в сообщаем, что IEK GROUP предоставляет компании ЭКС Новосибирск (филиал, Новосибирск, офис № 3 ) в рамках реализации проекта РФ Новосибирская обл Строительство ЖК "Расцветай" Новосибирск г Авиастроителей ул 3/1 стр право реализации продукции Торговой марки IEK по проекту 20240122-00561 ЖК Расцветай на Авиастроителей, ул.Авиастроителей, г.Новосибирск (ГК Расцветай) , указанной в предоставленной спецификации в адрес Алекс, ООО на следующих условиях (с предоставлением следующих скидок): Руководитель отдела продаж по СФО Плахотько К.А. Исх № P250721-40439 От 21.07.2025 ЭКС Новосибирск (филиал, Новосибирск, офис №3) Вниманию: - Исполнитель: Савинский Игорь Андреевич Тел: E-mail: savinskijia@iek.ru Сумма заказа 1 128 885,42 рублей с учетом предоставленных спец. условий. Перечень и количество продукции указаны в Приложении Также обращаю Ваше внимание на необходимость сразу же после каждо

In [94]:
# Вытаскиваем DiscountEKS – наша скидка
discount_from_base = "Устанавливаемая скидка (скидка от базовой), %"
# Поиск индексов (строка, столбец), где встречается текст
result = np.where(specification_xlsx.apply(lambda x: x.astype(str).str.contains(discount_from_base, regex=False)))
row_idx = result[0][0]  # Индекс строки
col_idx = result[1][0]  # Индекс столбца
DiscountEKS = float(specification_xlsx.iat[row_idx+1, col_idx])
print(DiscountEKS)

57.0


In [95]:
# Вытаскиваем DiscountBuyer – скидка клиента
text_discount_index = text.index('По продукции IEK - скидка до ')
text_discount = text[text_discount_index+29:text_discount_index+40]
text_discount = text_discount[:text_discount.index('%')]
text_discount = text_discount.replace(',', '.')
DiscountBuyer = float(text_discount)
print(DiscountBuyer)

55.0


In [96]:
# Вытаскиваем VendorCode – код вендора
vendor_code = "Номер проекта в системе IEK"
# Поиск индексов (строка, столбец), где встречается текст
result = np.where(specification_xlsx.apply(lambda x: x.astype(str).str.contains(vendor_code, regex=False)))
row_idx = result[0][0]  # Индекс строки
col_idx = result[1][0]  # Индекс столбца
VendorCode = specification_xlsx.iat[row_idx, col_idx+1]
print(VendorCode)

20240122-00561


In [97]:
# Вытаскиваем ReceivedCondition – сумма полученных условий
text_order_amount_index = text.index('Сумма заказа ')
text_order_amount = text[text_order_amount_index+12:text_order_amount_index+45]
text_order_amount = text_order_amount[:text_order_amount.index('рублей с учетом')]
text_order_amount = text_order_amount.replace(' ', '')
text_order_amount = text_order_amount.replace(',', '.')
ReceivedCondition = float(text_order_amount)
print(ReceivedCondition)
# print(text_order_amount)

1128885.42


In [98]:
# Вытаскиваем DateCondition – до какой даты действуют
date_condition = "Конец действия цен"
# Поиск индексов (строка, столбец), где встречается текст
result = np.where(specification_xlsx.apply(lambda x: x.astype(str).str.contains(date_condition, regex=False)))
row_idx = result[0][0]  # Индекс строки
col_idx = result[1][0]  # Индекс столбца
DateCondition = specification_xlsx.iat[row_idx, col_idx+1]
DateCondition = str(DateCondition)
print(DateCondition)

2025-09-20 12:58:19


In [99]:
# Данные для сохранения

Preference = 1

data = {
    "DiscountEKS": DiscountEKS,
    "DiscountBuyer": DiscountBuyer,
    "Preference": Preference,
    "VendorCode": VendorCode,
    "ReceivedCondition": ReceivedCondition,
    "DateCondition": DateCondition,
    "PathSupplierLetter": PathSupplierLetter
}

# Укажите путь, куда сохранить файл
json_file_path = "data.json"  # Замените на свой путь

# Сохраняем данные в JSON-файл
with open(json_file_path, "w", encoding="utf-8") as file:
    json.dump(data, file, indent=4, ensure_ascii=False)  # indent для красивого форматирования